# feasibility map

This Jupyter notebook is a brief walkthrough covering core functionalities of the feasibility map. For a full up-to-date documentation, please refer to the [feasibility map GitHub repository](https://github.com/HUISUN24/feasibility_map.git)

### Install feasibility map

Installing feasibility map is simple and easy utilizing either **PyPI** package repository or cloning from **GitHub**.
While not required, it is recommended to first set up a virtual environment using venv or Conda. This ensures that
one of the required versions of Python (3.9) is used and there are no dependency conflicts. If you have Conda
installed on your system, you can create a new environment with:

    conda create -n feasibility_map python=3.9
    conda activate feasibility_map

And then simply install feasibility_map from PyPI with

    pip install feasibility_map

Alternatively, you can also install feasibility map in editable mode if you cloned it from GitHub like

    git clone https://github.com/HUISUN24/feasibility_map.git

Or by downloading a ZIP file. Please note, this will by default download the latest development version of the
software, which may not be stable. For a stable version, you can specify a version tag after the URL with
`--branch <tag_name> --single-branch`.

Then, move to the feasibility_map folder and install in editable (`-e`) mode

    cd feasibility_map
    pip install -e .
    
### Install scheil
One of the dependency need fix by cloning from **GitHub**.

    git clone https://github.com/HUISUN24/scheil.git
    cd scheil
    pip install -e .

### then run pytest to test the functions

    pytest

### Starting with feasibility_map

In [1]:
import os
import time
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from fmat.core.compositions import generateCompositions,createComposition
from fmat.ref_data import periodic_table,materials
from fmat.core.pycalphad_run import pycalphad_eq,pycalphad_scheil
from fmat.core.GenerateEqScript import createEqScript
from fmat.core.ReadEqResult import getEqdata
from fmat.core.GenerateScheilScript import createScheilScript
from fmat.core.ReadScheilResult import getScheilSolidPhase
from fmat.plot.FeasibilityMap import plotMaps

Materials info

Input

In [2]:
# Create Compositions
comps = ['SS304L','NiCr','V']
database = 'TCFE8'
eleAmountType = 'massFraction'
pressure = 101325
ngridpts = 41  # number of points along each dimension of the composition grid
TemperatureRange = (900, 2300,10) #(lower limit, upper limit, temperature step)
indep_comps = [comps[1], comps[2]]  # choose them automatically
for i in comps:
    if i in periodic_table:
        materials[i] = {i:1}
    elif i not in materials.keys():
        materials['SS304L'] = {'Ni':0.09611451943, 'Cr':0.1993865031, 'Fe':0.7044989775}         # the composition of this element/alloys(in weight fractions)
maxNumSim = 250  # maximum number of simulations in each TCM file
# Equilibrium simulation settings

 # 'C-Cr-Cu-Fe-Mo-Nb-Ni-03-08.tdb' #'C-Cr-Cu-Fe-Mo-Nb-Ni-10-05.tdb'#'Cr-Fe-Ni-Ti-V_04-05.tdb' # <userDatabase>.TDB or TCFE8
eleAmountType = 'massFraction' # Candidates: massFraction massPercent moleFraction molePercent
output_Eq = f'{TemperatureRange[0]}-{TemperatureRange[1]}-{TemperatureRange[2]}-{comps[0]}-{comps[1]}-{comps[2]}-Eq'

In [3]:
from datetime import datetime
current_dateTime = datetime.now()
if '.tdb' in database or '.TDB' in database:
    database_name = database.split('/')
    path = f'./Simulation/{datetime.now().strftime("%m-%d-%Y")}-{comps[0]}-{comps[1]}-{comps[2]}-database-{database_name[-1][:-4]}'
else:
    path = f'./Simulation/{datetime.now().strftime("%m-%d-%Y")}-{comps[0]}-{comps[1]}-{comps[2]}-database-{database}'
isExist = os.path.exists(path)
if not isExist:
    os.makedirs(path)
    print("The new directory is created!")

The new directory is created!


In [27]:
compositions_list = generateCompositions(indep_comps,ngridpts)
Compositions, numPoint, comp, numSimultion = createComposition(indep_comps,comps,compositions_list,materials,path)
settings = [TemperatureRange,numPoint,numSimultion,comp,comps,indep_comps,database,pressure,eleAmountType]
print(settings)
np.save(f'{path}/setting.npy',settings)

Equilibrium simulation #: 903
Point #: 903
Element #: 4
relatedElement #: ['V', 'FE', 'CR', 'NI']
[(900, 2300, 10), 903, 903, ['V', 'FE', 'CR', 'NI'], ['SS304L', 'NiCr', 'V'], ['NiCr', 'V'], 'TCFE8', 101325, 'massFraction']


ValueError: setting an array element with a sequence. The requested array has an inhomogeneous shape after 1 dimensions. The detected shape was (9,) + inhomogeneous part.

Running with pycalphad

In [35]:
pycalphad_eq(path)
pycalphad_scheil(path,2000)

{X_AL: 3.997628572726799e-05, X_CU: 1.6974162677322643e-05} (1/3)
{X_AL: 2.3551034016150884e-05, X_CU: 0.9999705579325001} (2/3)
{X_AL: 0.9999932524620158, X_CU: 4.246114245598749e-06} (3/3)


/Users/sunhui/opt/anaconda3/envs/feasibility1/lib/python3.9/site-packages/pycalphad/core/utils.py:55: RuntimeWarning: invalid value encountered in divide
  pts[:, cur_idx:end_idx] /= pts[:, cur_idx:end_idx].sum(axis=1)[:, None]
/Users/sunhui/opt/anaconda3/envs/feasibility1/lib/python3.9/site-packages/pycalphad/core/utils.py:55: RuntimeWarning: invalid value encountered in divide
  pts[:, cur_idx:end_idx] /= pts[:, cur_idx:end_idx].sum(axis=1)[:, None]
/Users/sunhui/opt/anaconda3/envs/feasibility1/lib/python3.9/site-packages/pycalphad/core/utils.py:55: RuntimeWarning: invalid value encountered in divide
  pts[:, cur_idx:end_idx] /= pts[:, cur_idx:end_idx].sum(axis=1)[:, None]


The new directory is created!
cannot find liquid phase in eq results, start with back up temperature
cannot find liquid phase in eq results, start with back up temperature
cannot find liquid phase in eq results, start with back up temperature
{X_AL: 3.997628572726799e-05, X_CU: 1.6974162677322643e-05} (1/3)
{X_AL: 2.3551034016150884e-05, X_CU: 0.9999705579325001} (2/3)
{X_AL: 0.9999932524620158, X_CU: 4.246114245598749e-06} (3/3)


/Users/sunhui/opt/anaconda3/envs/feasibility1/lib/python3.9/site-packages/pycalphad/core/utils.py:157: FutureWarning: generate_dof is deprecated and will be removed in a future version of pycalphad. The correct way to determine the degrees of freedom of a particular 'active' phase is to use Model.constituents.
  warnings.warn(msg, FutureWarning)
/Users/sunhui/opt/anaconda3/envs/feasibility1/lib/python3.9/site-packages/pycalphad/core/utils.py:157: FutureWarning: generate_dof is deprecated and will be removed in a future version of pycalphad. The correct way to determine the degrees of freedom of a particular 'active' phase is to use Model.constituents.
  warnings.warn(msg, FutureWarning)
/Users/sunhui/opt/anaconda3/envs/feasibility1/lib/python3.9/site-packages/pycalphad/core/utils.py:157: FutureWarning: generate_dof is deprecated and will be removed in a future version of pycalphad. The correct way to determine the degrees of freedom of a particular 'active' phase is to use Model.consti

The new directory is created!


Running with Thermo_Calc

In [6]:
createEqScript(path)
createScheilScript(path,2000)

EOFError: No data left in file

Get results with Thermo_Calc

In [6]:
getEqdata(path)
getScheilSolidPhase(path)

###################################################################
####################### Reading Eq Result #########################


100%|██████████| 3/3 [00:00<00:00, 11.24it/s]


####################### Reading Eq Result Done ####################
0 files failed. FailList:
[]
#####################################################
#############start reading Scheil Result#############


100%|██████████| 3/3 [00:00<00:00, 619.33it/s]


####################################################################
Getting final Scheil result...


100%|██████████| 3/3 [00:00<00:00, 43539.49it/s]

##########################end reading Scheil#############################
Read Scheil Temperature vs Liquid fraction (mole):
Failed Index List: []
0 Files Failed to Read


Plot deleterious phase diagram and crack susceptibility map

In [7]:
plotMaps('./Simulation/5-31-2023-Fe-Cr-Ni-FEDEMO','thermocalc')

FileNotFoundError: [Errno 2] No such file or directory: '/Users/sunhui/Desktop/new-charge/feasibility_map/tasks/Simulation/5-31-2023-Fe-Cr-Ni-FEDEMO/Thermo-calc/Scheil Simulation/Result/data_mole.json'